In [1]:
import matplotlib as mpl
import os
import glob
import json
import pymworks
import re
import datautils
import copy
import math
import time

import multiprocessing as mp
import numpy as np
import pandas as pd
import seaborn as sns
import pylab as pl
import cPickle as pkl
from cPickle import PicklingError

import scipy.stats as spstats
import utils as util
import process_datafiles as processd

import pprint

pp = pprint.PrettyPrinter(indent=4)

def atoi(text):
    return int(text) if text.isdigit() else text
def natural_keys(text):
    return [ atoi(c) for c in re.split('(\d+)', text) ]


/home/julianarhee/anaconda2/envs/behavior/lib/python2.7/site-packages/pymworks/protocol/__init__.py:15: UserWarning: networkx is required for pymworks.protocols.states
  warnings.warn('networkx is required for pymworks.protocols.states')
/home/julianarhee/anaconda2/envs/behavior/lib/python2.7/site-packages/datautils/__init__.py:22: UserWarning: datautils.mongo failed to import with: No module named pymongo
  warnings.warn('datautils.mongo failed to import with: %s' % E)
utils.py:4: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/home/julianarhee/anaconda2/envs/behavior/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/home/julianarhee/anaconda2/e

# Set source and get meta info

In [2]:
rootdir = '/n/coxfs01/behavior-data'
paradigm = 'threeport'

metadata = util.get_metadata(paradigm, rootdir=rootdir, create_meta=False)


Loading existing metadata...


In [250]:
 np.linspace(-60, 60, 25.)

/home/julianarhee/anaconda2/envs/behavior/lib/python2.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: object of type <type 'float'> cannot be safely interpreted as an integer.
  """Entry point for launching an IPython kernel.


array([-60., -55., -50., -45., -40., -35., -30., -25., -20., -15., -10.,
        -5.,   0.,   5.,  10.,  15.,  20.,  25.,  30.,  35.,  40.,  45.,
        50.,  55.,  60.])

In [258]:
np.linspace(15, 40, 6.)

/home/julianarhee/anaconda2/envs/behavior/lib/python2.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: object of type <type 'float'> cannot be safely interpreted as an integer.
  """Entry point for launching an IPython kernel.


array([15., 20., 25., 30., 35., 40.])

# Assign phases to each datafile

In [467]:
cohort = 'AK'

In [468]:
phase_lookup = {0: 'manual',
                1: 'default',
                2: 'size',
                3: 'depth_rotation',
                4: 'cross',
                5: 'size_and_depth_rotation',
                6: 'depth_and_planar_rotation',
                7: 'morph',
                8: 'newstimuli',
                9: 'fine_grained_size',
                10: 'fine_grained_depth_rotation',
                11: 'fine_grained_size_and_depth_rotation',
                12: 'transparency',
                13: 'clutter',
                -1: 'other'}

default_size = 30
default_depth_rotation = 0.
default_planar_rotation = 0.

if cohort in ['AK', 'AL', 'AM']:
    expected_sizes = np.linspace(15, 40, 11.)
    expected_drots = np.linspace(-60, 60, 25.)
    default_size = 40
    
elif cohort in ['AG', 'AJ']:
    expected_sizes = np.linspace(15, 40, 6.)
    expected_drots = np.linspace(-60, 60, 9.)


expected_size_interval = np.diff(expected_sizes).mean()
expected_drot_interval = np.diff(expected_drots).mean()


new_stim_descs = ['stimB', 'stimC', 'NewObjects']


# array(['', 'b', 'big', 'stimB', 'stimBlowerbound', 'stimClowerbound',
#        'stimBupperbound', 'stimCupperbound', 'small', 'stimC', 'c',
#        'stimCalwaysReward', 'alwaysReward', 'size', 'adeptrotl',
#        'bdeptrotr', 'deptrotl', 'adeptrotr', 'asize', 'bdeptrotl',
#        'background', 'deptrotr', 'balwaysReward', 'initial', 'deprotl',
#        'backgroundalwaysReward', 'bsize', 'morphs', 'aalwaysReward',
#        'empty', 'aasize', 'a', 'probeNewObjects', 'occluded', 'r', 'd',
#        'transparency', 'nominrt'], dtype=object)


/home/julianarhee/anaconda2/envs/behavior/lib/python2.7/site-packages/ipykernel_launcher.py:22: DeprecationWarning: object of type <type 'float'> cannot be safely interpreted as an integer.
/home/julianarhee/anaconda2/envs/behavior/lib/python2.7/site-packages/ipykernel_launcher.py:23: DeprecationWarning: object of type <type 'float'> cannot be safely interpreted as an integer.


In [469]:
nfiles = len([f for f, g in animal_meta.groupby(['animalid', 'datasource'])])


In [470]:
expected_drot_interval, expected_size_interval

(5.0, 2.5)

In [495]:
exclude_ixs = []
phasedata = []
animal_meta = metadata[metadata['cohort']== cohort]

for (animalid, dfn), mgroup in animal_meta.sort_values(by=['animalid', 'session']).groupby(['animalid', 'datasource']):
    
    phase = -1
    (animalid, cohort, dsource, session, sfx), = mgroup.values

    curr_trials, curr_flags, metainfo = util.parse_mw_file(dfn, create_new=False)
    
    if curr_trials is None or len(curr_trials)==0:
        exclude_ixs.extend(mgroup.index.tolist())
        continue
        
    sizes = sorted(np.unique([t['size'] for t in curr_trials]))
    drots = sorted(np.unique([t['depth_rotation'] for t in curr_trials]))
    prots = sorted(np.unique([t['rotation'] for t in curr_trials]))
    
    tested_size_interval = np.diff(sizes).mean()
    tested_drot_interval = np.diff(drots).mean()

    protocol = metainfo['protocol']
    experiment_folder = metainfo['experiment']
    
    alpha_values = []
    if any(['alpha_multiplier' in s.keys() for s in curr_trials]):
        alpha_values = np.unique([s['alpha_multiplier'] for s in curr_trials])
    
#     if session == 20180625:
#         break

    if 'morph' in protocol:
        phase = 7
    
    elif 'newstim' in metainfo['experiment'] or any([descr in sfx for descr in new_stim_descs]):
        phase = 8

    elif len(alpha_values) > 1 or any([a != 1 for a in alpha_values]):
        phase = 12

    elif any(['background' in desc for desc in [sfx, metainfo['experiment']]]):
        phase = 13
    
    elif 'occluded' in sfx:
        phase = 13
    
    # ====== PHASE 1 ====================================
    elif len(sizes) == 1 and len(drots) == 1 and len(prots)==1:
        #if protocol in ['Initiate VIsual Pres Protocol', 'Initiate Visual Pres Protocol', '']:
        if (sizes[0]==default_size and drots[0]==default_depth_rotation and prots[0]==default_planar_rotation):
            phase = 1
    
    # ====== PHASE 2 ====================================
    elif len(sizes) > 1 and (len(drots)==1 and len(prots)==1):
        #if metainfo['protocol'] in ['Staircase through shape parameters', '']:        
        if ( (expected_sizes==sizes) is False):
            if (tested_size_interval < expected_size_interval):
                phase = 9
            elif (tested_size_interval==expected_size_interval):
                if (len(sizes) > len(expected_sizes)-1):
                    # Probably just screwed up indexing
                    phase = 2
                elif all([d in expected_sizes for d in sizes]) : 
                    # only testing a subset of the expected values
                    phase = 2
        elif all(expected_sizes==sizes):
            phase = 2
            
    # ====== PHASE 3 ====================================
    elif len(drots) > 1 and (len(sizes)==1 and len(prots)==1):
        #if metainfo['protocol'] in ['Staircase through shape parameters', '']:
        if ( (expected_drots==drots) is False):
            if (tested_drot_interval < expected_drot_interval):
                # Fine-grained spacing
                phase = 10
            elif (tested_drot_interval==expected_drot_interval):
                if (len(drots) > len(expected_drots)-1):
                    # Probably just screwed up indexing
                    phase = 3
                elif all([d in expected_drots for d in drots]): 
                    # only testing a subset of the expected values
                    phase = 3
        elif all(expected_drots==drots):
            phase = 3
            
    # ====== PHASE 4/5 ====================================
    elif len(drots) > 1 and len(sizes) > 1 and len(prots) == 1:
        #if metainfo['protocol'] in ['Test all transformations', '']:
        off_cross_transforms = list(set([(t['size'], t['depth_rotation']) for t in curr_trials \
                                         if t['size']!=default_size \
                                         and t['depth_rotation']!=default_depth_rotation]))
        
        if ( (expected_drots==drots) is False) or ( (expected_sizes==sizes) is False):
            if (tested_size_interval < expected_size_interval) or (tested_drot_interval < expected_drot_interval):
                # Fine-grained size/rotation transformations
                phase = 11
#             elif (len(drots) == len(expected_drots)-1) or (len(sizes) == len(expected_sizes)-1):
#                 # Probably just screwed up indexing
#                 if len(off_cross_transforms) == 0:
#                     phase = 4 if len(off_cross_transforms) == 0 else 5
#             elif ((expected_drots==drots) is False):
#                 if all(sizes==expected_sizes):
#                     # Another screw up, just not changing 1 of the depth-rot sides
#                     phase = 4 if len(off_cross_transforms)==0 else 5
#             elif ((expected_sizes==sizes) is False):
#                 if all(drots==expected_drots):
#                     # Another screw up, just not changing 1 of the sizes
#                     phase = 4 if len(off_cross_transforms)==0 else 5       
            elif all([d in expected_drots for d in drots]) and all([s in expected_sizes for s in sizes]):
                phase = 4 if (len(off_cross_transforms)==0) else 5
        else:
            # STANDARD
            if all(drots==expected_drots) and all(sizes==expected_sizes):
                phase = 4 if (len(off_cross_transforms)==0) else 5

    # ====== PHASE 6 (in-plane) ====================================
    elif len(prots) > 1 and len(sizes)==1:
        if metainfo['protocol'] == 'Test all transformations':
            phase = 6
    
    else:
        print("NO CLUE.")
        print("[%s, %s%s] unknown protocol: %s" % (animalid, session, suffix, protocol) )

        print("Size", sizes)
        print("D-rots", drots)
        print("P-rots", prots)
    
    print(animalid, session, phase)
    mgroup['phase'] = [phase for _ in np.arange(0, len(mgroup))]
    mgroup['protocol'] = [protocol for _ in np.arange(0, len(mgroup))]
    mgroup['experiment'] = [experiment_folder for _ in np.arange(0, len(mgroup))]
    
    phasedata.append(mgroup)
    
phasedata = pd.concat(phasedata, axis=0)


***** Parsing trials *****
-- saving tmp outfile to: /n/coxfs01/behavior-data/threeport/cohort_data/AK/processed/tmp_files/AK1_170528.pkl
-- finding stop
N total response events:  0
N total outcome events:  0
Found and removed 0 orphan stimulus events in file /n/coxfs01/behavior-data/threeport/cohort_data/AK/raw/AK1_170528.mwk
N valid trials: 0


/home/julianarhee/anaconda2/envs/behavior/lib/python2.7/site-packages/ipykernel_launcher.py:20: RuntimeWarning: Mean of empty slice.
/home/julianarhee/anaconda2/envs/behavior/lib/python2.7/site-packages/ipykernel_launcher.py:21: RuntimeWarning: Mean of empty slice.
/home/julianarhee/anaconda2/envs/behavior/lib/python2.7/site-packages/ipykernel_launcher.py:129: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/julianarhee/anaconda2/envs/behavior/lib/python2.7/site-packages/ipykernel_launcher.py:130: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-cop

('AK1', 20170531, 1)
('AK1', 20170606, 1)
('AK1', 20170607, 1)
('AK1', 20170608, 1)
('AK1', 20170609, 1)
('AK1', 20170612, 1)
('AK1', 20170613, 1)
('AK1', 20170614, 1)


/home/julianarhee/anaconda2/envs/behavior/lib/python2.7/site-packages/ipykernel_launcher.py:57: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.


('AK1', 20170615, 2)
('AK1', 20170616, 2)


...

-- finding stop
N total response events:  0
N total outcome events:  0
Found and removed 0 orphan stimulus events in file /n/coxfs01/behavior-data/threeport/cohort_data/AK/raw/AK4_170802.mwk
N valid trials: 0
('AK4', 20170803, 5)
('AK4', 20170804, 5)
('AK4', 20170807, 5)
('AK4', 20170808, 5)
('AK4', 20170809, 5)
('AK4', 20170810, 5)
('AK4', 20170811, 5)
('AK4', 20170814, 5)
('AK4', 20170815, 5)
('AK4', 20170816, 5)
('AK4', 20170822, 5)
('AK4', 20170823, 5)
('AK4', 20170908, 5)
('AK4', 20170911, 5)
('AK4', 20170912, 5)
('AK4', 20170913, 5)
('AK4', 20170919, 5)
('AK4', 20170920, 5)
('AK4', 20170921, 5)
('AK4', 20170922, 5)
('AK4', 20170925, 5)
('AK4', 20170926, 5)
('AK4', 20170927, 5)
('AK4', 20170928, 5)
('AK4', 20170929, 5)
('AK4', 20171002, 5)
('AK4', 20171003, 5)
('AK4', 20171004, 5)
('AK4', 20171005, 5)
('AK4', 20171006, 5)
('AK4', 20171010, 5)
('AK4', 20171107, 5)
('AK4', 20171108, 5)
('AK4', 20171109, 5)
('AK4', 20171110, 5)
('AK4', 20171111, 5)
('AK4', 20171112, 5)
('AK4', 201711

***** Parsing trials *****
-- saving tmp outfile to: /n/coxfs01/behavior-data/threeport/cohort_data/AK/processed/tmp_files/AK5_170911.pkl
here


--- unable to open file: /n/coxfs01/behavior-data/threeport/cohort_data/AK/raw/AK5_170911.mwk
ABORTING.
('AK5', 20170912, 5)
('AK5', 20170913, 5)
('AK5', 20170919, 5)
('AK5', 20170920, 5)
('AK5', 20170921, 5)
('AK5', 20170922, 5)
('AK5', 20170925, 5)
('AK5', 20170926, 5)
('AK5', 20170927, 5)
('AK5', 20170928, 5)
('AK5', 20170929, 5)
('AK5', 20171002, 5)
('AK5', 20171003, 5)
('AK5', 20171004, 5)
('AK5', 20171005, 5)
('AK5', 20171006, 5)
('AK5', 20171010, 5)
('AK5', 20171107, 5)
('AK5', 20171108, 5)
('AK5', 20171109, 5)
('AK5', 20171110, 5)
('AK5', 20171111, 5)
('AK5', 20171112, 5)
('AK5', 20171113, 5)
('AK5', 20171116, 5)
('AK5', 20171117, 5)
('AK5', 20171120, 5)
('AK5', 20171130, 5)
('AK5', 20171201, 5)
('AK5', 20171205, 5)
('AK5', 20171206, 5)
('AK5', 20171207, 5)
('AK5', 20171208, 5)
('AK5', 20171209, -1)
('AK5', 20171210, 5)
('AK5', 20171211, 5)
('AK5', 20171212, 5)
('AK5', 20171213, 5)
('AK5', 20171214, 5)
('AK5', 20180206, 5)
('AK5', 20180207, 5)
('AK5', 20180208, 5)
('AK5', 20180

here


--- unable to open file: /n/coxfs01/behavior-data/threeport/cohort_data/AK/raw/AK7_170907.mwk
ABORTING.
('AK7', 20170908, 5)
('AK7', 20170911, 5)
('AK7', 20170912, 5)
('AK7', 20170913, 5)
('AK7', 20170919, 5)
('AK7', 20170920, 5)
('AK7', 20170921, 5)
('AK7', 20170922, 5)
('AK7', 20170925, 5)
('AK7', 20170926, 5)
('AK7', 20170927, 5)
('AK7', 20170928, 5)
('AK7', 20170929, 5)
('AK7', 20171002, 5)
('AK7', 20171003, 5)
('AK7', 20171004, 5)
('AK7', 20171005, 5)
('AK7', 20171006, 5)
('AK7', 20171010, 5)
('AK7', 20171107, 5)
('AK7', 20171108, 5)
('AK7', 20171109, 5)
('AK7', 20171110, 5)
('AK7', 20171111, 5)
('AK7', 20171112, 5)
('AK7', 20171113, 5)
('AK7', 20171116, 5)
('AK7', 20171117, 5)
('AK7', 20171120, 5)
('AK7', 20171130, 5)
('AK7', 20171201, 5)
('AK7', 20171205, 5)
('AK7', 20171206, 5)
('AK7', 20171207, 5)
('AK7', 20171208, 5)
('AK7', 20171209, -1)
('AK7', 20171210, 5)
('AK7', 20171211, 5)
('AK7', 20171212, 5)
('AK7', 20171213, 5)
('AK7', 20171214, 5)
('AK7', 20180206, 5)
('AK7', 20180

In [497]:
phase_dfile = os.path.join(rootdir, paradigm, 'processed', 'meta', 'phases_%s.pkl' % cohort)
with open(phase_dfile, 'wb') as f:
    pkl.dump(phasedata, f, protocol=pkl.HIGHEST_PROTOCOL)
        

In [500]:
phase_dfile

'/n/coxfs01/behavior-data/threeport/processed/meta/phases_AK.pkl'

In [498]:
phasedata['phase'].unique()

array([ 1,  2,  3,  5, -1, 12])

In [499]:
phasedata.groupby(['animalid', 'phase']).count()

cohort  datasource  session  suffix  protocol  experiment
animalid phase                                                           
AK1      -1          1           1        1       1         1           1
          1          8           8        8       8         8           8
          2          2           2        2       2         2           2
          3          2           2        2       2         2           2
          5         90          90       90      90        90          90
          12        11          11       11      11        11          11
AK10      1         11          11       11      11        11          11
          2          3           3        3       3         3           3
          3         10          10       10      10        10          10
          5         88          88       88      88        88          88
AK11      1         14          14       14      14        14          14
          2          2           2        2       2         2           2
          3          4           4        4       4         4           4
          5         91          91       91      91        91          91
AK12     -1          1           1        1       1         1           1
          1          8           8        8       8         8           8
          2          2           2        2       2         2           2
          3          2           2        2       2         2           2
          5         91          91       91      91        91          91
AK2      -1          2           2        2       2         2           2
          1         18          18       18      18        18          18
          2          7           7        7       7         7           7
          3          5           5        5       5         5           5
          5         75          75       75      75        75          75
          12        11          11       11      11        11          11
AK3      -1          4           4        4       4         4           4
          1          7           7        7       7         7           7
          2          1           1        1       1         1           1
          3          1           1        1       1         1           1
          5         92          92       92      92        92          92
...                ...         ...      ...     ...       ...         ...
AK4       1          9           9        9       9         9           9
          2          3           3        3       3         3           3
          3          1           1        1       1         1           1
          5         92          92       92      92        92          92
AK5      -1          1           1        1       1         1           1
          1         10          10       10      10        10          10
          2          1           1        1       1         1           1
          3          2           2        2       2         2           2
          5         87          87       87      87        87          87
          12        11          11       11      11        11          11
AK6      -1          2           2        2       2         2           2
          1         18          18       18      18        18          18
          2          2           2        2       2         2           2
          3          7           7        7       7         7           7
          5         70          70       70      70        70          70
          12        11          11       11      11        11          11
AK7      -1          3           3        3       3         3           3
          1         14          14       14      14        14          14
          2          2           2        2       2         2           2
          3          4           4        4       4         4           4
          5         80          80       80      80        80          80
          12        

In [481]:
protocol

'Test all transformations'

In [313]:
[d in expected_drots for d in drots[0:5]]

[True, True, True, True, True]

In [315]:
off_cross_transforms = list(set([(t['size'], t['depth_rotation']) for t in curr_trials \
                                 if (t['size']!=default_size \
                                 and t['depth_rotation']!=default_depth_rotation) ]))
off_cross_transforms

[(30.0, 15),
 (30.0, -30),
 (30.0, 60),
 (30.0, -15),
 (30.0, 30),
 (30.0, -45),
 (30.0, -60),
 (30.0, 45)]

In [350]:
S.source

['/n/coxfs01/behavior-data/threeport/cohort_data/AL/raw/AL1_171130.mwk']

In [351]:
d = pymworks.open(S.source[0])

In [352]:
d.get_events('LickOutput1')

[Event[code=259, name=LickOutput1, time=2004829261699, value=0.1],
 Event[code=259, name=LickOutput1, time=2004830108115, value=0.1],
 Event[code=259, name=LickOutput1, time=2006428601444, value=0.1],
 Event[code=259, name=LickOutput1, time=2007051796978, value=0.01],
 Event[code=259, name=LickOutput1, time=2007121613847, value=0.01],
 Event[code=259, name=LickOutput1, time=2007182037724, value=0.01],
 Event[code=259, name=LickOutput1, time=2007189519851, value=0.01],
 Event[code=259, name=LickOutput1, time=2007206210890, value=0.01],
 Event[code=259, name=LickOutput1, time=2007227324080, value=0.01],
 Event[code=259, name=LickOutput1, time=2007236653259, value=0.01],
 Event[code=259, name=LickOutput1, time=2007251942368, value=0.01],
 Event[code=259, name=LickOutput1, time=2007284010807, value=0.01],
 Event[code=259, name=LickOutput1, time=2007300261724, value=0.01],
 Event[code=259, name=LickOutput1, time=2007307203121, value=0.01],
 Event[code=259, name=LickOutput1, time=20073298266

In [308]:
phase = 4 if len(off_cross_transforms) == 0 else 5
phase

4

In [232]:
phasedata[phasedata['phase']==-1].groupby(['protocol', 'experiment']).count()

animalid  \
protocol                           experiment                                                     
                                   _Users_labuser_Downloads_RatBehaviorGeneral_Beh...         2   
Initiate Visual Pres Protocol      _Users_labuser_Documents_MWorks_behavior_protoc...         9   
                                   _Users_labuser_Downloads_RatBehaviorGeneral_Beh...       240   
                                   _Users_labuser_Downloads_RatBehaviorGeneral_Beh...       104   
Staircase through shape parameters _Users_labuser_Downloads_RatBehaviorGeneral_Beh...         6   
Test all transformations           _Users_labuser_Documents_MWorks_behavior_protoc...       126   
                                   _Users_labuser_Downloads_RatBehaviorGeneral_Beh...       100   
                                   _Users_labuser_Downloads_RatBehaviorGeneral_Beh...         1   

                                                                                       cohort  \
protocol                           experiment                                                   
                                   _Users_labuser_Downloads_RatBehaviorGeneral_Beh...       2   
Initiate Visual Pres Protocol      _Users_labuser_Documents_MWorks_behavior_protoc...       9   
                                   _Users_labuser_Downloads_RatBehaviorGeneral_Beh...     240   
                                   _Users_labuser_Downloads_RatBehaviorGeneral_Beh...     104   
Staircase through shape parameters _Users_labuser_Downloads_RatBehaviorGeneral_Beh...       6   
Test all transformations           _Users_labuser_Documents_MWorks_behavior_protoc...     126   
                                   _Users_labuser_Downloads_RatBehaviorGeneral_Beh...     100   
                                   _Users_labuser_Downloads_RatBehaviorGeneral_Beh...       1   

                                                                                       datasource  \
protocol                           experiment                                                       
                                   _Users_labuser_Downloads_RatBehaviorGeneral_Beh...           2   
Initiate Visual Pres Protocol      _Users_labuser_Documents_MWorks_behavior_protoc...           9   
                                   _Users_labuser_Downloads_RatBehaviorGeneral_Beh...         240   
                                   _Users_labuser_Downloads_RatBehaviorGeneral_Beh...         104   
Staircase through shape parameters _Users_labuser_Downloads_RatBehaviorGeneral_Beh...           6   
Test all transformations           _Users_labuser_Documents_MWorks_behavior_protoc...         126   
                                   _Users_labuser_Downloads_RatBehaviorGeneral_Beh...         100   
                                   _Users_labuser_Downloads_RatBehaviorGeneral_Beh...           1   

                                                                                       session  \
protocol                           experiment                                                    
                                   _Users_labuser_Downloads_RatBehaviorGeneral_Beh...        2   
Initiate Visual Pres Protocol      _Users_labuser_Documents_MWorks_behavior_protoc...        9   
                                   _Users_labuser_Downloads_RatBehaviorGeneral_Beh...      240   
                                   _Users_labuser_Downloads_RatBehaviorGeneral_Beh...      104   
Staircase through shape parameters _Users_labuser_Downloads_RatBehaviorGeneral_Beh...        6   
Test all transformations           _Users_labuser_Documents_MWorks_behavior_protoc...      126   
                                   _Users_labuser_Downloads_RatBehaviorGeneral_Beh...      100   
                                   _Users_labuser_Downloads_RatBehaviorGeneral_Beh...        1   

                                                                                       suffix  \
protocol                           e

In [ ]:
# check if this is in excluded:
# ABORTED:  /n/coxfs01/behavior-data/threeport/cohort_data/AL/raw/AL4_170919.mwk
# 


In [7]:
include_ixs = [i for i in animal_meta.index.tolist() if i not in exclude_ixs]
print(len(exclude_ixs), len(include_ixs), animal_meta.shape)


(13, 1498, (1511, 5))


In [26]:
def check_against_manual_sorting(animalid, phase, paradigm='threeport', 
                                 rootdir='/n/coxfs01/behavior-data'):
    '''
    Based on JM's sorting, expects:
        PHASE:
            2_size
            3_deptrotr_l
            4_fullcross, etc.
            
        os.path.join(by_phase_cohort_dir, 'AL13 - AL16', <PHASE>, AL*_YYMMDD.mwk)
    
    '''
    
    included_sessions =  None
    sorted_dfns = None
    
    cohort = re.search('(\D+)', animalid).group(0)
    sorted_cohort_dir = os.path.join(rootdir, paradigm, 'by_cohort', '%s_Cohort' % cohort, 'sorted', '_data')

    if cohort in ['AM', 'AN']:
        # 'PHASE_description'
        sorted_dfns = sorted(glob.glob(os.path.join(sorted_cohort_dir, '%i_*' % phase, \
                                                    '%s*' % cohort, '%s_*.mwk' % animalid)), key=natural_keys)
    elif cohort == 'AL':
        sorted_dfns = sorted(glob.glob(os.path.join(rootdir, paradigm, 'by_phase', '%s*' % cohort, 
                                                          '%i_*' % phase, '%s*' % cohort, 
                                                          '%s_*.mwk' % animalid)), key=natural_keys)
    elif cohort == 'AK':
        sorted_dfns = sorted(glob.glob(os.path.join(sorted_cohort_dir, '_regular*', '%i_*' % phase, \
                                                    '%s*' % cohort, '%s_*.mwk' % animalid)), key=natural_keys)
    
    elif cohort == 'AJ':
        sorted_dfns = sorted(glob.glob(os.path.join(rootdir, paradigm, 'by_cohort', '%s_Cohort' % cohort, 'phase%i' % phase,
                                         '%s*' % cohort, '%s_*.mwk' % animalid)), key=natural_keys)
    elif cohort == 'AG':
        if phase in [3, 4]:
            sorted_dfns = glob.glob(os.path.join(rootdir, paradigm, 'by_cohort', '%s_Cohort' % cohort,
                                     '%s phase%i' % (cohort, phase), '%s*' % cohort, '%s_*.mwk' % animalid))
        else:
            return None

    #print("[%s] %i datafiles for phase %i" % (animalid, len(sorted_dfns), phase))
    if sorted_dfns is not None:
        included_sessions = [str(i) for i in \
                             sorted(list(set([util.parse_datafile_name(dfn)[1] for dfn in sorted_dfns])), 
                                    key=natural_keys)]

    return included_sessions



In [233]:
cohort

'AL'

In [234]:
phase = 3
cohort = 'AL'

# excluded_animals = {3: ['AG12']}


check_these = dict((animalid, {}) for animalid in phasedata['animalid'].unique())
not_found = []
for animalid, pdata in phasedata[phasedata['phase']==phase].groupby(['animalid']):
    
    manual_list = check_against_manual_sorting(animalid, phase)
    if manual_list is None:
        not_found.append((phase, animalid))
    
#     manual_list = sorted([str('20%s' % os.path.splitext(os.path.split(fn)[1])[0].split('_')[-1]) \
#                           for fn in manual_sorted_dfiles], key=natural_keys)
    
    found_list = sorted(['%s%s' % (session, suffix) for (session, suffix), g in pdata.groupby(['session', 'suffix'])], key=natural_keys)
    print("%s: Manual %i, Auto %i " % (animalid, len(manual_list), len(found_list)))
    
    if manual_list != found_list:
        check_these[animalid].update({'not_in_manual': [i for i in found_list if i not in manual_list],
                                      'not_in_found': [i for i in manual_list if i not in found_list]})
        

In [235]:
not_found

[]

In [236]:
check_these

{'AL1': {},
 'AL10': {},
 'AL11': {},
 'AL12': {},
 'AL13': {},
 'AL14': {},
 'AL15': {},
 'AL16': {},
 'AL2': {},
 'AL3': {},
 'AL4': {},
 'AL5': {},
 'AL6': {},
 'AL7': {},
 'AL8': {},
 'AL9': {}}

In [164]:
animalid = 'AL7'
# check_sessions = ['20170912'] 
check_sessions = check_these[animalid]['not_in_found'] # Not in auto-sorted

check_pdata = phasedata[ (phasedata['animalid']==animalid) & (phasedata['session'].isin(check_sessions))].copy()
for (prot, exp), g in check_pdata.groupby(['protocol', 'experiment']):
    if 'morph' in prot:
        print prot

exp = '_Users_labuser_Downloads_RatBehaviorGeneral_Behavior_3Dtransforms_2way_blobs'
check_pdata[check_pdata['experiment']==exp]['session'].unique()


array([20170727])

In [165]:
print(animalid)
check_these[animalid]

AL7


{'not_in_found': ['20170727'], 'not_in_manual': ['20170727b', '20170728b']}

In [166]:
check_pdata

,animalid,cohort,datasource,session,suffix,phase,protocol,experiment
1231,AL7,AL,/n/coxfs01/behavior-data/threeport/cohort_data...,20170727,,2,Staircase through shape parameters,_Users_labuser_Downloads_RatBehaviorGeneral_Be...
1230,AL7,AL,/n/coxfs01/behavior-data/threeport/cohort_data...,20170727,b,3,Staircase through shape parameters,_Users_labuser_Downloads_RatBehaviorGeneral_Be...


In [455]:
metadata['suffix'].unique()

array(['', 'b', 'big', 'stimB', 'stimBlowerbound', 'stimClowerbound',
       'stimBupperbound', 'stimCupperbound', 'small', 'stimC', 'c',
       'stimCalwaysReward', 'alwaysReward', 'size', 'adeptrotl',
       'bdeptrotr', 'deptrotl', 'adeptrotr', 'asize', 'bdeptrotl',
       'background', 'deptrotr', 'balwaysReward', 'initial', 'deprotl',
       'backgroundalwaysReward', 'bsize', 'morphs', 'aalwaysReward',
       'empty', 'aasize', 'a', 'probeNewObjects', 'occluded', 'r', 'd',
       'transparency', 'nominrt'], dtype=object)

In [490]:

animalid = 'AK1'
session = 20180623 #0627 #0606

session_meta = metadata[(metadata['animalid']==animalid) & (metadata['session']==session)]
S = util.Session(session_meta)
_, _, metainfo = S.get_trials()
metainfo

--> [1] of 1: /n/coxfs01/behavior-data/threeport/cohort_data/AK/raw/AK1_180623.mwk


{'address': '192.168.0.36',
 'experiment': '_Users_labuser_Documents_MWorks_behavior_protocols_Behavior_3Dtransforms_2way_blobs',
 'experiment_path': '/var/folders/ct/9rl6yygn7474gnc092xp2mzc0000gp/T/MWorks/Experiment Cache/_Users_labuser_Documents_MWorks_behavior_protocols_Behavior_3Dtransforms_2way_blobs',
 'protocol': 'Test all transformations',
 'screen': {'distance': 12.5,
  'height': 11.75,
  'refresh_rate_hz': 60.0,
  'units': 'inches',
  'width': 14.75},
 'server': 'Setup 18'}

In [491]:
suffix_values = np.unique([s['suffix'] for s in S.trials])
suffix_values

array([''], dtype='|S1')

In [492]:
for si, sfx in enumerate(suffix_values):
    
    print("%i of %i -- %s" % (int(si+1), len(suffix_values), sfx))
    
    curr_trials = [s for s in S.trials if s['suffix']==sfx]

    off_cross_transforms = list(set([(t['size'], t['depth_rotation']) for t in curr_trials \
                                     if t['size']!=default_size \
                                     and t['depth_rotation']!=default_depth_rotation]))
    print("off-cross:", off_cross_transforms)
    print("---")

    sizes = sorted(np.unique([t['size'] for t in S.trials]))
    drots = sorted(np.unique([t['depth_rotation'] for t in S.trials]))
    prots = sorted(np.unique([t['rotation'] for t in S.trials]))

    print("sizes:", sizes)
    print("depth rots:", drots)
    print("planar rots:", prots)
    
    print("---")

    #phasedata[ (phasedata['animalid']==animalid) & (phasedata['session']==int(session)) ]

1 of 1 -- 
('off-cross:', [(30.0, 15), (30.0, -30), (30.0, 60), (30.0, -15), (30.0, 30), (30.0, -45), (30.0, -60), (30.0, 45)])
---
('sizes:', [15.0, 20.0, 25.0, 30.0, 35.0, 40.0])
('depth rots:', [-60, -45, -30, -15, 0, 15, 30, 45, 60])
('planar rots:', [0.0])
---


In [493]:
if any(['alpha_multiplier' in s.keys() for s in S.trials]):
    alpha_values = [s['alpha_multiplier'] for s in S.trials]

np.unique(alpha_values)

array([1.])

In [494]:
S.trials[0]

{'action': 'draw',
 'alpha_multiplier': 1.0,
 'depth_rotation': -60,
 'duration': 383185,
 'file_hash': '332a7afdb7fdd025f644c3546d98017ad57f8dc2',
 'filename': '/var/folders/ct/9rl6yygn7474gnc092xp2mzc0000gp/T/MWorks/Experiment Cache/_Users_labuser_Documents_MWorks_behavior_protocols_Behavior_3Dtransforms_2way_blobs/tmp/Blobs_TrainingRatsD1D2/Blob_N2_CamRot_y-60.png',
 'name': 'Blob_N2_CamRot_y-60',
 'no_feedback': False,
 'object': '2',
 'outcome': 'failure',
 'outcome_time': 5785235414970,
 'pos_x': 0.0,
 'pos_y': 0.0,
 'response': 'Announce_AcquirePort3',
 'response_time': 5785235413881,
 'rotation': 0.0,
 'size': 30.0,
 'size_x': 52.40999984741211,
 'size_y': 52.40999984741211,
 'suffix': '',
 'time': 5785235069946,
 'type': 'image'}

In [440]:
metadata[ (metadata['animalid']==animalid) & (metadata['session']==int(session)) ]

,animalid,cohort,datasource,session,suffix
1717,AM1,AM,/n/coxfs01/behavior-data/threeport/cohort_data...,20180411,background


In [441]:
phasedata.head()

AttributeError: 'list' object has no attribute 'head'

In [478]:
alpha_values = np.unique([s['alpha_multiplier'] for s in S.trials])
if len(alpha_values) > 1:
    phase = 12
elif any([a!=1 for a in alpha_values]):
    phase = 12

In [479]:
phase

12